In [1]:
# https://rdrr.io/cran/grf/man/grf.html
# install.packages('grf')
#install.packages('ATE')
#install.packages("tidyverse")

In [2]:
library(grf)
library(ATE)
#library(tidyverse)

In [3]:
size = 5000

In [4]:
# Treatment
set.seed(12)
data <- runif(size)
Treatment <- ifelse(data<0.5,0,1)
Age <- round(data*100)
b_Age <- ifelse(Age >=40 & Age <= 60, 1, 0)

In [5]:
# Y ~ T, no confounding factor
# effectsize = 0.05
Y <- 0.05 * Treatment + 0.05 * rnorm(size)
l1 <- lm(Y~Treatment)
summary(l1)$coef
ATE(Y, Treatment,matrix(runif(size)),theta=1)

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),0.0004104584,0.001011389,0.4058365,6.848801e-01
Treatment,0.0486458297,0.001425198,34.1326754,1.018875e-229


Call:
ATE(Y = Y, Ti = Treatment, X = matrix(runif(size)), theta = 1)

The analysis was completed for a simple study design with binary treatment.

Point Estimates:
    E[Y(1)]     E[Y(0)]         ATE 
0.049053362 0.000386705 0.048666657 

In [6]:
# Y ~ T with HTE
Y <- 0.05 * Treatment * b_Age + 0.05 * rnorm(size)
l2 <- lm(Y ~ Treatment)
summary(l2)$coef
ATE(Y, Treatment,matrix(0.05 * rnorm(size)),theta=1)


,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-0.0003493102,0.001045143,-0.3342224,7.382258e-01
Treatment,0.0118518249,0.001472763,8.0473399,1.048426e-15


Call:
ATE(Y = Y, Ti = Treatment, X = matrix(0.05 * rnorm(size)), theta = 1)

The analysis was completed for a simple study design with binary treatment.

Point Estimates:
      E[Y(1)]       E[Y(0)]           ATE 
 0.0114772047 -0.0003318944  0.0118090991 

In [7]:
Y <- 0.05 * Treatment * b_Age + 0.05 * rnorm(size)
z <- 
l2 <- lm(Y ~ Treatment : b_Age)
summary(l2)$coef
ATE(Y, Treatment,matrix(Age),theta=1)

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),0.0008562634,0.0007430058,1.152432,2.491988e-01
Treatment:b_Age,0.0494236182,0.0022321445,22.141765,1.049937e-103


Call:
ATE(Y = Y, Ti = Treatment, X = matrix(Age), theta = 1)

The analysis was completed for a simple study design with binary treatment.

Point Estimates:
    E[Y(1)]     E[Y(0)]         ATE 
0.035753403 0.002004222 0.033749181 

In [8]:
cf <- causal_forest(
  X = matrix(Age),
  Y = Y,
  W = Treatment,
  num.trees = 5000,
  seed = 1839
)

In [9]:
average_treatment_effect(cf,target.sample = "overlap")

estimate    std.err 
0.05602397 0.01594405